In [1]:
import numpy as np

In [8]:
#matrix multiplication
def matrix_mult(X,Y):
    result = [[sum(a*b for a,b in zip(X_row,Y_col)) for Y_col in zip(*Y)] for X_row in X]
    return result

In [9]:
#matrix transpose
def matrix_trans(X):
    result = [[X[j][i] for j in range(len(X))] for i in range(len(X[0]))]
    return result

In [11]:
#matrix addition
def matrix_add(X,Y):
    result = [[X[i][j] + Y[i][j]  for j in range(len(X[0]))] for i in range(len(X))]
    return result

In [29]:
#matrix LU decomposition
def lu_decomposition(X):
        global n
        # Create matrices for L and U                                                                                                                                                                                                                 
        L = [[float(i ==j) for i in range(len(X))] for j in range(len(X))]    
        U = X
        
        for i in range(n):
            k = i + 1
            while (k < n):
                L_factor = U[k][i]/U[i][i]

                for j in range(n):
                    U[k][j] -= U[i][j]*L_factor
                L[k][i] = L_factor
                k+=1

        return L,U

In [127]:
#matrix inversion with LU decomposition
def lu_inverse(L,U):
    #create identity matrix
    id_mat = [[float(i ==j) for i in range(len(L))] for j in range(len(L))]
    #initialise inverse matrix
    A_inv = []
    #Let Z = U * Minverse, where M = L*U
    #Given L*Z=I, solve for Z with forward substitution
    print("Z-matrix (U * Minverse)")
    Z=[]
    MinvT=[]
    for cols in range(len(id_mat[0])):
        Zcol = []
        for i in range(len(id_mat)):
            linearsum = id_mat[i][cols]
            if i == 0:
                Zcol.append(linearsum/L[i][i])
            else:
                for j in range(i):
                    linearsum -= Zcol[j]*L[i][j]

                Zcol.append(linearsum/L[i][i])
        Z.append(Zcol)
    print(Z)
    print("\n")
    #Given U * Minverse = Z, solve for Minverse with backward substitution
    for cols in range(len(id_mat[0])):
            MinvTcol = [0.0 for i in range(len(Z))]
            for i in range(len(Z)-1,-1,-1):
                linearsum = Z[cols][i]
                if i == len(Z)-1:
                    MinvTcol[i]=linearsum/U[i][i]
                else:
                    for j in range(len(Z)-1,i-1,-1):
                        linearsum -= MinvTcol[j]*U[i][j]
                    MinvTcol[i] = linearsum/U[i][i]    
                
            MinvT.append((MinvTcol))
    Minv = matrix_trans(MinvT)
    print("Minverse, where M = ATA inverse + lambda*I")
    print(Minv)
    print("\n")
    return Minv

In [190]:
#input parameters: filepath, base number and lambda
def LSE(file,base,l):
    global n
    n = base
    data = np.loadtxt(file,delimiter=",")
    
    #print out the data points
    print ("Data points")
    print (data)
    print ("\n")
    
    #create design matrix
    X=np.empty([len(data),n])
    for i in range(len(data)):
        for j in range(n):
            X[i][j]= data[i][0]**j
    print("Design Matrix")
    print(X)
    print("\n")
    
    #calculate ATA
    ATA = matrix_mult(matrix_trans(X),X)
    print("ATA")
    print(ATA)
    print("\n")
    
    #add lambda*I for regularisation
    I=[[0 for x in range(n)] for y in range(n)]
    for i in range(0,n):
        I[i][i] = 1
    M = matrix_add(ATA,l*I)
    print("ATA + (Lambda * I)")
    print(M)
    print("\n")
    
    #LU Decomposition
    L,U = lu_decomposition(M)
    print("Upper Matrix")
    print(U)
    print("Lower Matrix")
    print(L)
    print("\n")
    
    #Calculate ATA inverse with LU
    ATAinv = lu_inverse(L,U)
    
    #create matrix y
    y=np.empty([len(data),1])
    for i in range(len(data)):
        y[i]=data[i][1]
    
    #let the correction vector be c
    c = matrix_mult(matrix_mult(ATAinv,matrix_trans(X)),y)
    
    print("The Least-Squared-Error regression line equation is ",end="", flush=True)
    for i in range(n):
        if(c[i][0]==0):
            continue
        if(c[i][0]==1):
            pass
        else:
            print(c[i][0],end="", flush=True)
        if(i==0):
            pass
        elif(i==1):
            print("x",end="", flush=True)
        elif(i==n-1):
            print("x^"+str(i))
            break
        else:
            print("x^"+str(i),end="", flush=True)
        print(" + ",end="", flush=True)
        
    #calculate error
    #let the error function be J(c) = (1/2m) * [(Xc-y)^2 + lambda*(c**2)]
    J = 0
    for i in range(len(X)):
        h=0
        for j in range(len(X[i])):
            h += c[j][0]*(X[i][j])
        J += (h-y[i])**2
    print ("The unregularised error function evaluates to be "+str(J/(2*len(X))))
    
    for i in c:
        J += l * (i[0]**2)
    print ("The regularised error function evaluates to be "+str(J/(2*len(X))))

        

In [191]:
LSE("data.txt",3,1)

Data points
[[ 0.  0.]
 [ 1.  1.]
 [-1.  1.]
 [ 2.  4.]
 [-2.  4.]
 [ 3.  9.]
 [-3.  9.]
 [ 5. 25.]
 [-5. 25.]]


Design Matrix
[[ 1.  0.  0.]
 [ 1.  1.  1.]
 [ 1. -1.  1.]
 [ 1.  2.  4.]
 [ 1. -2.  4.]
 [ 1.  3.  9.]
 [ 1. -3.  9.]
 [ 1.  5. 25.]
 [ 1. -5. 25.]]


ATA
[[9.0, 0.0, 78.0], [0.0, 78.0, 0.0], [78.0, 0.0, 1446.0]]


ATA + (Lambda * I)
[[10.0, 0.0, 78.0], [0.0, 79.0, 0.0], [78.0, 0.0, 1447.0]]


Upper Matrix
[[10.0, 0.0, 78.0], [0.0, 79.0, 0.0], [0.0, 0.0, 838.6]]
Lower Matrix
[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [7.8, 0.0, 1.0]]


Z-matrix (U * Minverse)
[[1.0, 0.0, -7.8], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]


Minverse, where M = ATA inverse + lambda*I
[[0.17254948724063918, 0.0, -0.009301216312902457], [0.0, 0.012658227848101266, 0.0], [-0.009301216312902457, 0.0, 0.0011924636298592892]]


The Least-Squared-Error regression line equation is 0.009301216312902927 + 0.9988075363701406x^2
The unregularised error function evaluates to be [6.13627242e-05]
The regularised error funct

In [218]:
#Newton's method
#Reference from https://www.youtube.com/watch?v=U6mwqyJzImg
def newton(file,n,l):
        #get data matrix as above
        data = np.loadtxt(file,delimiter=",")
        m = data
        x =[]
        y=[]
        for i in range(len(m)):
            x.append(m[i][0])
            y.append(m[i][1])
        n = len(x)
        q = np.zeros((n, n - 1))
        
        # Insert 'y' in the first column of the matrix 'q'
        for i in range(n):
            q[i][0] = y[i]
    
        for i in range(1, n-1):
            for j in range(1, i + 1):
                q[i, j] = (q[i, j - 1] - q[i - 1, j - 1]) / (x[i] - x[i - j])
            
        f = np.zeros(n)
        for i in range(0, n-1):
            f[i] = q[i, i]
    
        # Prints the polynomial
        print("The equation acquired from Newthon's Method of optimisation is ", end = "", flush = True) 
        print("y= %f" % f[0], end="")
        for i in range(1, n):
            if f[i] == 0:
                pass
            else:
                print(" + %f" % f[i], end="")
                for j in range(0, i ):
                    if(x[j]!=0):
                        print("(x%f)" % (x[j] * -1), end="")
                    else:
                        print("(x)", end="")
        print("\n")        
        
newton("data.txt",3,1)

The equation acquired from Newthon's Method of optimisation is y= 0.000000 + 1.000000(x) + 1.000000(x)(x-1.000000)

